# POLI 175 - Lecture 08

## Classification

## Classification

Loading packages:

In [ ]:
# If needed
using Pkg
Pkg.add("Missings")
Pkg.add("StatsAPI")
Pkg.add("FreqTables")
Pkg.add("EvalMetrics")
Pkg.update()

## Classification

Loading packages:

In [ ]:
## Loading the packages
using CSV, DataFrames, Plots, GLM, StatsBase, Random
using LaTeXStrings, StatsPlots, Lowess, Gadfly, RegressionTables
using CovarianceMatrices, Econometrics, LinearAlgebra, MixedModelsExtras
using Missings, StatsAPI, FreqTables, EvalMetrics

# Auxiliar function
function pairplot(df)
    _, cols = size(df)
    plots = []
    for row = 1:cols, col = 1:cols
        push!(
            plots,
            scatter(
                df[:, row],
                df[:, col],
                xtickfont = font(4),
                ytickfont = font(4),
                legend = false,
            ),
        )
    end
    Plots.plot(plots..., layout = (cols, cols))
end

# Classification

## Classification

Linear regression is great! But it assumes we want to predict a continuous target variable.

There are situations when our target is qualitative.

**Examples:**

1. Whether a country default its debt obligations?

1. Whether a person voted Republican, Democrat, Independent, voted for a different party, or did not turnout to vote?

1. What determines the number of FOI requests that a given public office receives every day?

1. Is a country expected to meet, exceed, or not meet the Paris Treaty Nationally Determined Contributions?

All these questions are qualitative in nature.

## Running Example

### [Chile Survey](https://en.wikipedia.org/wiki/Chile)

In 1988, the [Chilean Dictator](https://en.wikipedia.org/wiki/Military_dictatorship_of_Chile) [Augusto Pinochet](https://en.wikipedia.org/wiki/Augusto_Pinochet) conducted a [referendum to whether he should step out](https://en.wikipedia.org/wiki/1988_Chilean_presidential_referendum).

The [FLACSO](https://en.wikipedia.org/wiki/Latin_American_Faculty_of_Social_Sciences) in Chile conducted a surver on 2700 respondents. We are going to build a model to predict their voting intentions.

| **Variable** | **Meaning** |
|:---:|---|
| region | A factor with levels:<br>- `C`, Central; <br>- `M`, Metropolitan Santiago area; <br>- `N`, North; <br>- `S`, South; <br>- `SA`, city of Santiago. |
| population | The population size of respondent's community. |
| sex | A factor with levels: <br>- `F`, female; <br>- `M`, male. |
| age | The respondent's age in years. |
| education | A factor with levels: <br>- `P`, Primary; <br>- `S`, Secondary; <br>- `PS`, Post-secondary. |
| income | The respondent's monthly income, in Pesos. |
| statusquo | A scale of support for the status-quo. |
| vote | A factor with levels: <br>- `A`, will abstain; <br>- `N`, will vote no (against Pinochet);<br>- `U`, is undecided; <br>- `Y`, will vote yes (for Pinochet). |

## Chile Survey

In [ ]:
## Loading the data
chile = CSV.read(
    download("https://raw.githubusercontent.com/umbertomig/POLI175julia/main/data/chilesurvey.csv"), 
    DataFrame,
    missingstring = ["NA"]
); dropmissing!(chile)
chile.voteyes = ifelse.(chile.vote .== "Y", 1, 0)
first(chile, 3)

## Book's Example

Probability of voting `yes` (favor of Pinochet remain in power for eight more years), conditional on how satisfied the person is with things as they are:

In [ ]:
dat = DataFrame(statusquo = collect(range(-3, stop = 3, length = 100)));

In [ ]:
Gadfly.plot(
    x = chile.statusquo, y = chile.voteyes, Stat.y_jitter(range = 0.2), Geom.point,
    Guide.xlabel("Status Quo Preference"), Guide.ylabel("Vote for Pinochet"),
    layer(x = dat.statusquo, y = predict(glm(@formula(voteyes ~ statusquo), chile, Binomial(), LogitLink()), dat), Geom.line, color = [colorant"red"], order = 2),
    layer(x = dat.statusquo, y = predict(lm(@formula(voteyes ~ statusquo), chile), dat), Geom.line, color = [colorant"blue"], order = 3)
)

## Logistic Regression

[Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) belongs to a class of models called [Generalized Linear Models](https://en.wikipedia.org/wiki/Generalized_linear_model) (or GLM for short).

GLM, in a nutshell (and in a proudly lazy definition) is an expansion of Linear Model that assumes:
- A Linear Relationship in part of the model
- But then applies a non-linear transformation to the response variable.

The non-linear transformation is called `link function`. Many link functions around (check [here](https://en.wikipedia.org/wiki/Generalized_linear_model) for various link functions).

The link function will determine which types of models we run.

When the outcome variable is binary, we may use `Logistic` or `Probit` links.

## Logistic Regression

In a regression, we are investigating something along the lines of:

$$ \mathbb{E}[Y | X] \ = \ \beta_0 + \beta_1 X $$

But when the outcome is binary we would like to get:

$$ \mathbb{E}[Y | X] \ = \ \mathbb{P}(Y = 1 | X) $$

And the Logistic link is nothing but:

$$ \mathbb{P}(Y = 1 | X) \ = \ \dfrac{e^{(\beta_0 + \beta_1X)}}{1 + e^{(\beta_0 + \beta_1X)}} \ = \ \dfrac{1}{1 + e^{-(\beta_0 + \beta_1X)}} $$

## Logistic Regression

With a bit of manipulation, we get to something called odds ratio:

$$ \dfrac{\mathbb{P}(Y = 1 | X)}{\mathbb{P}(Y = 0 | X)} \ = \ \dfrac{\mathbb{P}(Y = 1 | X)}{1 - \mathbb{P}(Y = 1 | X)} \ = \ e^{(\beta_0 + \beta_1X)} $$

And logging the thing gets rid of the Euler constant:

$$ \log \left( \dfrac{\mathbb{P}(Y = 1 | X)}{1 - \mathbb{P}(Y = 1 | X)}\right) \ = \ \beta_0 + \beta_1X $$

And this is the [Logit Link](https://en.wikipedia.org/wiki/Logistic_regression).

## Logistic Regression

Little detour to talk about odd ratios:

- Note the odd ratio: $\dfrac{\mathbb{P}(Y = 1 | X)}{1 - \mathbb{P}(Y = 1 | X)}$

- It is a ratio between the chance of $Y = 1$ divided by the chance of $Y = 0$.

- Since probabilities are between zero and one, the ratio is always between $(0, \infty)$.

Example:

- If based on characteristics, two in every ten people vote for Pinochet, $\mathbb{P}(Y = 1 | X = \text{some characs.}) = 0.2$ and the odds ratio is $1/4$.

- If based on other set of characteristics, nine out of ten people vote for Pinochet, $\mathbb{P}(Y = 1 | X = \text{some other characs.}) = 0.9$ and the odds ratio is $9$.

One is the number that *does not change the ratios*.


## Logistic Regression

Little other detour to talk about the coefficients:

In linear regression, changes in one unit of $x_i$ changes your target variable in $\beta_i$ units, on average.

In logistic regression, changes in one unit of $x_i$ changes **the log odds** of your target variable in $\beta_i$ units, on average.

- Multiplies the odds by $e^{\beta_i}$! This is **not** a straight line!

- Easy proxy (does not work for interaction terms): 
    + When $\beta_1$ is **positive**, it **increases** the $\mathbb{P}(Y = 1 | X)$
    + When $\beta_1$ is **negative**, it **decreases** the $\mathbb{P}(Y = 1 | X)$
    
Try to compute the partial derivatives on $X$ and you will see the complications!

## Logistic Regression

Technical:

1. The estimation is through [maximizing the likelihood function](https://en.wikipedia.org/wiki/Likelihood_function).<br> This is outside the scope of the course, but an interesting topic to learn in an advanced course.<br> <br> 


2. The hypothesis test for the coefficient's significance in here is a Z-test (based on the Normal distribution). <br> Null Hypothesis: $$H_0: \beta_i = 0 \quad \text{ or alternatively, } \quad H_0: e^{\beta_i} = 1$$<br> <br> 


3. Making predictions: Just insert the predicted $\hat{\beta}$s on the equation.
    
$$ \hat{p}(X) \ = \ \dfrac{e^{\hat{\beta}_0 + \hat{\beta}_1 X}}{1 + e^{\hat{\beta}_0 + \hat{\beta}_1 X}} $$

## Logistic Regression

Fitting a Linear Regression using Julia GLM package. Note the syntax:

```julia
glm(
    @formula(target ~ feature1 + feature2 + ... + featureK), 
    dataset, 
    distr = Binomial(),
    link = LogitLink()
)
```

In [ ]:
modlog = glm(@formula(voteyes ~ age), chile, Binomial(), LogitLink())

What is happening here? Let us interpret some of the results.

## Logistic Regression

Compare with a linear regression:

In [ ]:
# Linear Model
modlin = lm(@formula(voteyes ~ age), chile)
regtable(modlin, modlog)

What is happening here? Let us interpret some of the results for both models.

## Logistic Regression

Column 1: [Linear Probability Model](https://en.wikipedia.org/wiki/Linear_probability_model)

Column 2: [Logistic Model](https://en.wikipedia.org/wiki/Logistic_regression)

Column 3: [Probit Model](https://en.wikipedia.org/wiki/Probit_model)

And there are other links that we may use.

In [ ]:
modpro = glm(@formula(voteyes ~ age), chile, Binomial(), ProbitLink())
regtable(modlin, modlog, modpro)

## Logistic Regression

We can easily fit multivariate regressions:

Model 1: [Linear Probability Model](https://en.wikipedia.org/wiki/Linear_probability_model)

Model 2: [Logistic Model](https://en.wikipedia.org/wiki/Logistic_regression)

Model 3: [Probit Model](https://en.wikipedia.org/wiki/Probit_model)

In [ ]:
modlin2 = lm(@formula(voteyes ~ region + log(population) + sex + age + education + log(income)), chile);
modlog2 = glm(@formula(voteyes ~ region + log(population) + sex + age + education + log(income)), chile, Binomial(), LogitLink());
modpro2 = glm(@formula(voteyes ~ region + log(population) + sex + age + education + log(income)), chile, Binomial(), ProbitLink());

## Logistic Regression

Now with a pretty table:

In [ ]:
regtable(modlin2, modlog2, modpro2; 
    groups = ["Linear", "Logistic", "Probit"],
    number_regressions = false,
    labels = Dict("voteyes" => "Vote for Pinochet", "(Intercept)" => "Constant", "region: M" => "Region = Santiago Metro",
        "region: N" => "Region = North", "region: S" => "Region = South", 
        "region: SA" => "Region = Santiago City", "log(population)" => "Log District Population",
        "sex: M" => "Male", "age" => "Age", "education: PS" => "Post-Secondary Education",
        "education: S" => "Secondary Education", "log(income)" => "Log District Income"
    )
)

## Logistic Regression

The predicted values are probabilities:

In [ ]:
Gadfly.plot(
    x = predict(modlog2), 
    Geom.histogram, 
    color = ifelse.(predict(modlog2).>0.5, "Predict Yes", "Predict No"),
    Guide.xlabel("Predicted Probabilities")
)

## Logistic Regression

Let's look at the coefficients. Those are log odds ratio:

In [ ]:
## Parameters
println(modlog2)
println(coef(modlog2))

## Logistic Regression

To make them odds ratio, we exponentiate.

In [ ]:
## Parameters
println(modlog2)
println(exp.(coef(modlog2)))

## Logistic Regression

**The interaction term**: 

We found in the previous models that:

1. The higher the district's population, the lower the chance of supporting Pinochet
1. The higher the district's income, the higher the chance of voting for Pinochet

The first result relates with people living in cities preferring democracy; the second results relate to people that are benefiting financially from the status quo preferring to keep things as they are.

But how about a rich person that lives in a city? In the US analogy, thing of billionaire democrat. She supports the democrats even though she may be benefiting from things they are.

Conversely, how about a poor person that lives in a village? It could be that the person is not benefiting from things they are, but she favors Pinochet for conservative issues.

Let us prepare a situation where population stays fixed on one of three values: 25th percentile (small villages), median (median-density districts), 75th percentile (more dense districts); and income varying from zero to one million pesos.

This should give an idea of how population and income jointly affect the probability of voting for Pinochet.

In [ ]:
# Auxiliary variables
chile.pop100k = chile.population / 100000;
chile.inc1mi = chile.income / 1000000;

In [ ]:
# Three interactions between a fixed income and a varying population
newdat_popq1 = DataFrame(
    pop100k = repeat([quantile(chile.pop100k, 0.25)], 100), 
    inc1mi = collect(range(0.0, stop = 1.0, length = 100))
);
newdat_popmed = DataFrame(
    pop100k = repeat([median(chile.pop100k)], 100), 
    inc1mi = collect(range(0.0, stop = 1.0, length = 100))
);
newdat_popq3 = DataFrame(
    pop100k = repeat([quantile(chile.pop100k, 0.75)], 100), 
    inc1mi = collect(range(0.0, stop = 1.0, length = 100))
);

## Logistic Regression

**The interaction term**: 

In a linear regression:

In [ ]:
## Parameters
modlin3 = lm(@formula(voteyes ~ pop100k * inc1mi ), chile)
regtable(modlin3; 
    number_regressions = false,
    labels = Dict(
        "voteyes" => "Vote for Pinochet", 
        "(Intercept)" => "Constant", 
        "pop100k" => "Population (100k people)",
        "inc1mi" => "Income (1 million pesos)",
        "pop100k & inc1mi" => "Population x Income"
    )
)

## Logistic Regression

**The interaction term**:

And here is how the interaction term affects the results:

In [ ]:
Gadfly.plot(x = newdat_popq1.inc1mi, y=predict(modlin3, newdat_popq1), Geom.line, color = ["Population (Q1)"],
    layer(x = newdat_popmed.inc1mi, y=predict(modlin3, newdat_popmed), Geom.line, color = ["Population (median)"]),
    layer(x = newdat_popq3.inc1mi, y=predict(modlin3, newdat_popq3), Geom.line, color = ["Population (Q3)"]),
    Guide.xlabel("Income"),
    Guide.ylabel("Chance of Voting for Pinochet"),
    Guide.title("Linear Probability Model") 
)

Note the ***linearity*** of the interaction effect. Note also the effects of extrapolating.

## Logistic Regression

**The interaction term**: 

In the logistic regression, you need to use a plot called [interactive effects plot](). If you are curious, [here is a good theoretical explanation, with Stata examples](https://stats.oarc.ucla.edu/stata/seminars/deciphering-interactions-in-logistic-regression/).

In [ ]:
## Parameters
modlog3 = glm(@formula(voteyes ~ pop100k * inc1mi), chile, Binomial(), LogitLink())
regtable(modlog3; 
    number_regressions = false,
    labels = Dict(
        "voteyes" => "Vote for Pinochet", 
        "(Intercept)" => "Constant", 
        "pop100k" => "Population (100k people)",
        "inc1mi" => "Income (1 million pesos)",
        "pop100k & inc1mi" => "Population x Income"
    )
)

## Logistic Regression

**The interaction term**:

Note now the ***non-linearity*** of the interaction effect:

In [ ]:
Gadfly.plot(x = newdat_popq1.inc1mi, y=predict(modlog3, newdat_popq1), Geom.line, color = ["Population (Q1)"],
    layer(x = newdat_popmed.inc1mi, y=predict(modlog3, newdat_popmed), Geom.line, color = ["Population (median)"]),
    layer(x = newdat_popq3.inc1mi, y=predict(modlog3, newdat_popq3), Geom.line, color = ["Population (Q3)"]),
    Guide.xlabel("Income"),
    Guide.ylabel("Chance of Voting for Pinochet"),
    Guide.title("Logistic Model") 
)

## Logistic Regression

In generalized linear models where the link function is not the identity function (i.e., not a linear regression), you need to be careful when interpreting and using interactions.

In linear models, it is straightforward.

In all models, it involves taking a partial derivative of the conditional expectation function that we fit in the training stage.

A catch is: This discussion matters more if you care about interpreting your model. For prediction, it does not matter that much.

***But what matters for prediction?***

## Logistic Regression

### Prediction

Suppose that we are an investment firm that will invest in Chile only if democracy wins.

Because the plebiscite did not happened, uncertainty may bring its own gains. Thus, it may be better to invest *before* the results.

All we have is this dataset. So, how much we predict modeling the vote for pinochet using these variables?

Let us model vote for Pinochet using the following variables: `log(income)`, `log(population)`, and `age`.

In [ ]:
modlog4 = glm(@formula(voteyes ~ log(population) + log(income) + age), chile, Binomial(), LogitLink())

How well is this model doing?

## Logistic Regression

### Prediction

We have significance, and we may think: pretty well! But like the $R^2$ for linear regressions, we need something that works similar in here.

Let us start checking how people voted:

In [ ]:
freqtable(chile.voteyes)

In [ ]:
proptable(chile.voteyes)

And if you were to guess, what would that guess be? ***This is our benchmark!***

## Logistic Regression

### Prediction

Now let us look at our model. 

For simplicity, we will say that if the model gives more than 0.5 chance of favoring Pinochet, the person voted in favor. Otherwise, the person voted against.

In [ ]:
Gadfly.plot(
    x = predict(modlog4), 
    Geom.histogram, 
    color = ifelse.(predict(modlog4).>0.5, "Predict Yes", "Predict No"),
    Guide.xlabel("Predicted Probabilities")
)

## Logistic Regression

### Prediction

Now let us look at our model. 

For simplicity, we will say that if the model gives more than 0.5 chance of favoring Pinochet, the person voted in favor. Otherwise, the person voted against.

In [ ]:
predicted_vote = ifelse.(predict(modlog4) .> 0.5, 1, 0);

In [ ]:
freqtable(predicted_vote)

In [ ]:
proptable(predicted_vote)

Are we doing better than chance? How can we know that?

## Logistic Regression

### Prediction

[**Confusion Matrix**](https://en.wikipedia.org/wiki/Confusion_matrix)

A confusion matrix is simply a display of the ground truth (rows) x the model's prediction (columns).

![confm](https://github.com/umbertomig/POLI175julia/blob/c9b0555e3e97778495bee72746aee43ddf3226d7/img/confm_wikipedia.png?raw=true)

In [ ]:
freqtable(chile.voteyes, predicted_vote)

### Prediction

[**Confusion Matrix**](https://en.wikipedia.org/wiki/Confusion_matrix)

|  | **Predicted: 0** | **Predicted: 1** |
|---|---|---|
| **Actual: 0** | True Negative (tn) | False Positive (fp) |
| **Actual: 1** | False Negative (fn) | True Positive (tp) |

[**Accuracy:**](https://en.wikipedia.org/wiki/Evaluation_of_binary_classifiers) $$\dfrac{\text{correct predictions}}{\text{total observations}} \ = \ \dfrac{tp + tn}{tp + tn + fp + fn}$$

- High accuracy: lots of correct predictions!

### Measuring Performance

[**Confusion Matrix**](https://en.wikipedia.org/wiki/Confusion_matrix)

|  | **Predicted: 0** | **Predicted: 1** |
|---|---|---|
| **Actual: 0** | True Negative (tn) | False Positive (fp) |
| **Actual: 1** | False Negative (fn) | True Positive (tp) |

[**Precision**](https://en.wikipedia.org/wiki/Precision_and_recall): $$\dfrac{\text{true positives}}{\text{total predicted positive}} \ = \ \dfrac{tp}{tp + fp}$$

- High precision: low false-positive rates.


### Measuring Performance

[**Confusion Matrix**](https://en.wikipedia.org/wiki/Confusion_matrix)

|  | **Predicted: 0** | **Predicted: 1** |
|---|---|---|
| **Actual: 0** | True Negative (tn) | False Positive (fp) |
| **Actual: 1** | False Negative (fn) | True Positive (tp) |

[**Recall**](https://en.wikipedia.org/wiki/Precision_and_recall): $$\dfrac{\text{true positives}}{\text{total actual positive}} \ = \ \dfrac{tp}{tp + fn}$$

- High recall: low false-negative rates.


### Measuring Performance

[**Confusion Matrix**](https://en.wikipedia.org/wiki/Confusion_matrix)

|  | **Predicted: 0** | **Predicted: 1** |
|---|---|---|
| **Actual: 0** | True Negative (tn) | False Positive (fp) |
| **Actual: 1** | False Negative (fn) | True Positive (tp) |

[**Precision**](https://en.wikipedia.org/wiki/Precision_and_recall): $$\dfrac{\text{true positives}}{\text{total predicted positive}} \ = \ \dfrac{tp}{tp + fp}$$

[**Recall**](https://en.wikipedia.org/wiki/Precision_and_recall): $$\dfrac{\text{true positives}}{\text{total actual positive}} \ = \ \dfrac{tp}{tp + fn}$$

[**F1-Score**](https://en.wikipedia.org/wiki/F-score):

$$ \text{F1} \ = \ 2 \times \dfrac{\text{precision} \times \text{recall}}{\text{precision} + \text{recall}}  \ = \ \dfrac{2 \ tp}{2 \ tp + fp + fn} $$

## Logistic Regression

### Prediction

[**Confusion Matrix**](https://en.wikipedia.org/wiki/Confusion_matrix)

In [ ]:
conf_mat = ConfusionMatrix(chile.voteyes, predicted_vote)

In [ ]:
accuracy(conf_mat)

In [ ]:
precision(conf_mat)

## Logistic Regression

### Prediction

[**Confusion Matrix**](https://en.wikipedia.org/wiki/Confusion_matrix)

In [ ]:
recall(conf_mat)

In [ ]:
f1_score(conf_mat)

## Logistic Regression

### Prediction

[**ROC Curve**](https://en.wikipedia.org/wiki/Receiver_operating_characteristic): Plots the true positive rates x false positive rates at each threshold.

The higher the area covered by the blue part (AUC, i.e., area under the curve), the better.

In [ ]:
rocplot(chile.voteyes, 
    predict(modlog4), 
    label = "vote ~ log(pop) + log(inc) + age", 
    diagonal = true)

## Why not run a Linear Regression?

You could ask this very valid question. And my answer here differs a bit from the book.

**Suggestion 01:**

If you want to **measure a treatment effect**, or any other fitting where **explanation trumps prediction**, go with the linear regression.
- Easy to explain to a lay audience.

- Good polynomial expansion around the ATE.

- But there is a need for a careful design (in Causal Inference, the design is more important than the statistical method!).

- Interaction terms are just partial derivatives of the fitted equation (no partial of the link function using chain rule).

## Why not run a Linear Regression?

**Suggestion 02:**

If you want to **predict outcomes**, go with a classification model appropriate for your target variable unit.

- You are not going to do `weird` prediction.

- You have a marginal efficiency gain (in terms of Standard Errors).

- If you have an ordered target variable, your model does *look like more meaningful*.

- You should be careful when interpreting interaction terms.

## Why not run a Linear Regression?

**Suggestion 03:**

Be **careful when you have discrete nominal variation in your target variable**:

- Binary outcome: Linear Regression and Linear Discriminant Analysis are the same.

- Three or more categories, like the `vote` in the Chilean dataset messes up badly with things.

# Questions?

# See you next class
